In [2]:
import numpy as np
import pandas as pd
import datetime
import sys, os

# Data parsing

In [14]:
# Parse html file
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 20
# MONTH = 5
# SUB = '/scrf'
SUB = ''
file_name = 'Оперативные данные _ Коронавирус COVID–19_ Официальная информация о коронавирусе в России на портале – стопкоронавирус.рф.html'

# Save of "https://xn--80aesfpebagmfblc0a.xn--p1ai/information/" page
path = f'html_data/{DAY}-{MONTH}{SUB}/{file_name}'
parse_df = pd.read_html(path, encoding='utf-8')[0]

parse_df.head()

,Регион,Выявлено,Новые,Активные ? Число больных в настоящее время,Выздоровело,Умерло
0,Москва,171443,2140,101802,67458,2183
1,Московская область,35956,793,29157,6440,359
2,Санкт-Петербург,14463,387,9847,4458,158
3,Нижегородская область,8735,231,5752,2905,78
4,Республика Дагестан,4455,161,164,4161,130


In [15]:
# Modifi data frame
parse_df = parse_df.drop(parse_df.columns[2:4] , axis=1)
parse_df.columns = ['Region/City', 'Confirmed', 'Recovered', 'Deaths']
parse_df['Date'] = datetime.datetime.now().strftime("%Y-%m-%d")
parse_df

,Region/City,Confirmed,Recovered,Deaths,Date
0,Москва,171443,67458,2183,2020-05-27
1,Московская область,35956,6440,359,2020-05-27
2,Санкт-Петербург,14463,4458,158,2020-05-27
3,Нижегородская область,8735,2905,78,2020-05-27
4,Республика Дагестан,4455,4161,130,2020-05-27
...,...,...,...,...,...
80,Севастополь,160,121,3,2020-05-27
81,Республика Алтай,105,53,0,2020-05-27
82,Сахалинская область,76,47,0,2020-05-27
83,Ненецкий автономный округ,66,12,0,2020-05-27


# Preparing parsed data

In [16]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
6560,2020-05-27,Челябинская область,Chelyabinsk region,74.0,70.0,0.0,64.0,2286.0,13.0,988.0
6561,2020-05-27,Чеченская Республика,Chechen Republic,95.0,19.0,0.0,8.0,1152.0,13.0,698.0
6562,2020-05-27,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,63.0,1.0,37.0
6563,2020-05-27,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,51.0,0.0,30.0,2202.0,6.0,788.0
6564,2020-05-27,Ярославская область,Yaroslavl region,76.0,82.0,0.0,47.0,2808.0,13.0,616.0


In [17]:
# Add day columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

# Rename regions
rename_dict = {
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [18]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [19]:
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Day-Confirmed'].sum()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Day-Deaths'].sum()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Day-Recovered'].sum()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [20]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [10]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 299941 Day-Confirmed: -70739.0
Russia Deaths: 2837 Day-Deaths: -1131.0
Russia Recovered: 76130 Day-Recovered: -66078.0


# Data saving

In [46]:
# Save full update table
parse_df.to_csv(f'covid19-russia-cases-upd-full-scrf.csv', index=False)

In [47]:
# Drop rows without day changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [48]:
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}-scrf.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd-scrf.csv', index=False)